In [ ]:
!pip install langchain langchain_ollama
!pip install chromadb sentence-transformers langchain_huggingface langchain_chroma
!pip install gradio
!pip install bs4


## Librerias que necesitamos

In [49]:
import requests
from bs4 import BeautifulSoup
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [50]:

url = 'https://www.rcdeportivo.es/anosahistoria'
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)

In [51]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,  
    chunk_overlap=100,  
    length_function=len
)
chunks = text_splitter.split_text(text)

In [52]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [53]:
from langchain_chroma import Chroma


vector_store = Chroma.from_texts(
    texts=chunks,
    collection_name="futbol",
    embedding=embeddings,
    persist_directory="./datasets"
)

In [54]:
results = vector_store.similarity_search_with_score(
    "The History of Deportivo de la Coruña", k=3,
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content}")


* [SIM=0.629696] RCDEPORTIVO
El RC Deportivo visita la exposición sobre la vida y obra de Cheché Martín
El Dépor, único equipo campeón de las tres primeras categorías del fútbol español
#OSNOSOS
Ignacio García Vázquez, coordinador del fútbol base y directivo del RC Deportivo entre 1986 y 1988, falleció a los 77 años de edad
#ANOSAHISTORIA
El RC Deportivo recibió en ABANCA-RIAZOR a Pauleta, campeón de Liga como blanquiazul en 2000
#ANOSAHISTORIA
Vegazo, extremo deportivista a mediados de los años 50, falleció a los 90 años
#ANOSAHISTORIA
Celestino Ledo, jugador deportivista en la temporada 1954-55, falleció a los 89 años
#ANOSAHISTORIA
Paco Lodeiro Barbeito, directivo deportivista entre 1982 y 1986, falleció hoy a los 85 años
#ANOSAHISTORIA
Cano, jugador del Dépor entre 1983 y 1985, falleció a los 67 años
#ANOSAHISTORIA
Fallece Luis 'el Chato', icono blanquiazul de los años 60 y 70
#ANOSAHISTORIA
Ver más
Aviso Legal y Condiciones de Uso
Política de cookies
Política de privacidad
Configu

In [55]:
retriever = vector_store.as_retriever(
    search_type="similarity",  search_kwargs={"k": 3}
)

In [61]:
query = "What is the nickname of Deportivo de La Coruña?"
results = retriever.get_relevant_documents(query)  

for i, result in enumerate(results):
    print(f"Resultado {i+1}:")
    print(result.page_content) 
    print()

Resultado 1:
Indicadores TI
Descargas
Contacto
Canal Ético
ES
#ANOSAHISTORIA
El RC Deportivo ha conquistado siete títulos oficiales a lo largo de su historia. Un Campeonato de Liga (1999-2000), una Copa España (1912), dos Copas del Rey (1995 y 2002) y tres Supercopas de España (1995, 2000 y 2002). El RC Deportivo es uno de los nueve clubes que ha sido capaz de vencer la Liga española, de la que además ha sido subcampeón en cinco ocasiones: 1949-1950, 1993-1994, 1994-1995, 2000-2001 y 2001-2002. También se proclamó campeón de la Segunda División en cinco ocasiones (1961-1962, 1963-1964, 1965-1966, 1967-1968 y 2011-2012), aunque en las cuatro primeras dicha categoría estaba dividida en dos grupos y el RC Deportivo fue primer clasificado del Grupo Norte. También fue campeón de Tercera División en la única campaña en la que militó en dicha categoría (1974-1975). En sus primeras décadas de vida, el RC Deportivo conquistó seis Campeonatos de Galicia, en 1927, 1928, 1931, 1933, 1937 y 1940.
R

In [57]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [63]:
chain.invoke("When did Deportivo de La Coruña win La Liga?")

'Deportivo de La Coruña ganó la Liga española en una ocasión, en la temporada 1999-2000.'

## Crear GUI

In [67]:
def search_chroma(query, top_k):
    try:   
        results = vector_store.similarity_search(query, k=top_k)
        return "\n\n".join(
            [f"**Result {i+1}:**\n{doc.page_content}" for i, doc in enumerate(results)]
        )
    except Exception as e:
        return f"Error: {e}"

In [68]:
import gradio as gr

In [66]:
with gr.Blocks() as demo:
    gr.Markdown("### Chroma Database Search")
    
    with gr.Row():
        query_input = gr.Textbox(label="Enter Your Query", placeholder="Type your question here...")
        top_k_input = gr.Slider(1, 10, step=1, value=5, label="Number of Results")

    search_button = gr.Button("Search")
    output_box = gr.Textbox(label="Search Results", lines=15)

    search_button.click(fn=search_chroma, inputs=[query_input, top_k_input], outputs=output_box)

demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
